In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

In [2]:
DATA_PATH = '../data/'
FEATURE_PATH = '../'
train_log_df = pd.read_csv(os.path.join(DATA_PATH, 'train_log.csv'))
metadata = train_log_df[['object_id', 'Z', 'EBV', 'target']]
print("Metadata loaded successfully")

Metadata loaded successfully


In [3]:
# Load ALL Lightcurve Data for tsFresh
all_lc_df_list = []
print("Loading all lightcurve data...")
for split_folder in tqdm(train_log_df['split'].unique()):
    path = os.path.join(DATA_PATH, split_folder, 'train_full_lightcurves.csv')
    df = pd.read_csv(path)
    all_lc_df_list.append(df)

full_lc_df = pd.concat(all_lc_df_list)
print(f"Full lightcurve data loaded. Shape: {full_lc_df.shape}")

Loading all lightcurve data...


100%|██████████| 20/20 [00:00<00:00, 94.79it/s]

Full lightcurve data loaded. Shape: (479384, 5)


In [4]:
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import EfficientFCParameters


# Feature Extraction with tsFresh
print("Starting tsFresh feature extraction per filter...")

band_features = []
for f in full_lc_df['Filter'].unique():
    subset = full_lc_df[full_lc_df['Filter'] == f][['object_id', 'Time (MJD)', 'Flux']].copy()
    subset.columns = ['id', 'time', 'value']
    
    # Drop NaNs
    subset = subset.dropna(subset=['value'])
    
    feats = extract_features(
        subset,
        column_id='id',
        column_sort='time',
        default_fc_parameters=EfficientFCParameters(),
        disable_progressbar=False,
        n_jobs=4
    )
    # tag features with filter name
    feats.columns = [f"{col}_{f}" for col in feats.columns]
    band_features.append(feats)

# Merge all filter-specific features
extracted_features = pd.concat(band_features, axis=1)

# Impute NaN/Infs
impute(extracted_features)

# Feature selection
y_for_selection = metadata.set_index('object_id').loc[extracted_features.index]['target']
relevant_features = select_features(
    extracted_features,
    y_for_selection,
    fdr_level=0.005
)

print(f"tsFresh finished successfully. Found {relevant_features.shape[1]} relevant features.")


Starting tsFresh feature extraction per filter...


Feature Extraction: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]
c:\Project\MALLORN Astronomical Classification Challenge\.venv\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['value__query_similarity_count__query_None__threshold_0.0_z'
 'value__query_similarity_count__query_None__threshold_0.0_r'
 'value__fft_coefficient__attr_"real"__coeff_93_y'
 'value__fft_coefficient__attr_"real"__coeff_94_y'
 'value__fft_coefficient__attr_"real"__coeff_95_y'
 'value__fft_coefficient__attr_"real"__coeff_96_y'
 'value__fft_coefficient__attr_"real"__coeff_97_y'
 'value__fft_coefficient__attr_"real"__coeff_98_y'
 'value__fft_coefficient__attr_"real"__coeff_99_y'
 'value__fft_coefficient__attr_"imag"__coeff_93_y'
 'value__fft_coefficient__attr_"imag"__coeff_94_y'
 'value__fft_coefficient__attr_"imag"__coeff_95_y'
 'value__fft_coefficient__attr_"imag"__coeff_96_y'
 'value__fft_coefficient__attr_"imag"__coeff_97_y'
 'value__fft_coefficient__attr_"imag"__co

tsFresh finished successfully. Found 198 relevant features.


In [5]:
# Combine Features and Prepare for Modeling
# Marge the new tsFresh with best old features
final_features = metadata.merge( 
    relevant_features,
    left_on='object_id',
    right_index=True,
    how='inner'
)

# Clean column names 
final_features.columns = (
    final_features.columns
    .str.replace('[^A-Za-z0-9_]+', '_', regex=True)  
    .str.strip('_')                                  
)

print("Saving final_features.csv...")
final_features.to_csv('final_features.csv', index=False)
print("File saved successfully.")

# Prepare X and y
y = final_features['target']
X = final_features.drop(columns=['object_id', 'target'])
features = X.columns.tolist()

print(f"Data prepared for final modeling. New features shape: {X.shape}")

Saving final_features.csv...
File saved successfully.
Data prepared for final modeling. New features shape: (3043, 200)


In [6]:
import optuna
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

# Standard scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

# Cross-validation
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Objective function for Optuna
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'n_estimators': 2000,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'verbose': -1
    }

    f1_scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        # Compute scale_pos_weight
        neg_count = y_train.value_counts()[0]
        pos_count = y_train.value_counts()[1]
        params['scale_pos_weight'] = neg_count / pos_count

        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='f1',
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
        )

        val_preds_proba = model.predict_proba(X_val)[:, 1]
        thresholds = np.linspace(0.01, 0.99, 100)
        f1_values = [f1_score(y_val, (val_preds_proba > t).astype(int)) for t in thresholds]
        f1_scores.append(np.max(f1_values))

    return np.mean(f1_scores)

# Run Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\nBest F1 Score:", study.best_value)
print("Best Hyperparameters:", study.best_params)


c:\Project\MALLORN Astronomical Classification Challenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-06 15:39:38,775] A new study created in memory with name: no-name-0de1a288-c1dd-4b71-bba7-0849fd28f094
Best trial: 0. Best value: 0.496018:   2%|▏         | 1/50 [00:14<11:44, 14.39s/it]

[I 2025-11-06 15:39:53,162] Trial 0 finished with value: 0.4960182279994023 and parameters: {'learning_rate': 0.02395753895685934, 'num_leaves': 57, 'max_depth': 8, 'min_child_samples': 57, 'subsample': 0.844336096890923, 'colsample_bytree': 0.638007240637462, 'reg_alpha': 0.10007190678904954, 'reg_lambda': 0.1021877360137542}. Best is trial 0 with value: 0.4960182279994023.


Best trial: 0. Best value: 0.496018:   4%|▍         | 2/50 [00:21<08:03, 10.07s/it]

[I 2025-11-06 15:40:00,215] Trial 1 finished with value: 0.49183944037426397 and parameters: {'learning_rate': 0.061022691141937674, 'num_leaves': 48, 'max_depth': 15, 'min_child_samples': 62, 'subsample': 0.7242533844054863, 'colsample_bytree': 0.8630972288835417, 'reg_alpha': 0.9413421325213572, 'reg_lambda': 0.9333634510889371}. Best is trial 0 with value: 0.4960182279994023.


Best trial: 0. Best value: 0.496018:   6%|▌         | 3/50 [00:30<07:23,  9.44s/it]

...

[I 2025-11-06 15:47:19,134] Trial 47 finished with value: 0.46959654489385505 and parameters: {'learning_rate': 0.017510241941768097, 'num_leaves': 27, 'max_depth': 9, 'min_child_samples': 56, 'subsample': 0.9056675170648367, 'colsample_bytree': 0.9289996348125511, 'reg_alpha': 0.3369041723607979, 'reg_lambda': 0.4719261990227097}. Best is trial 34 with value: 0.5371970983176417.


Best trial: 34. Best value: 0.537197:  98%|█████████▊| 49/50 [07:51<00:11, 11.48s/it]

[I 2025-11-06 15:47:30,387] Trial 48 finished with value: 0.4667633347045112 and parameters: {'learning_rate': 0.010730385765350344, 'num_leaves': 35, 'max_depth': 12, 'min_child_samples': 68, 'subsample': 0.949965625015573, 'colsample_bytree': 0.5994083586704654, 'reg_alpha': 0.47855626825285824, 'reg_lambda': 0.8952834631863896}. Best is trial 34 with value: 0.5371970983176417.


Best trial: 34. Best value: 0.537197: 100%|██████████| 50/50 [08:03<00:00,  9.67s/it]

[I 2025-11-06 15:47:42,374] Trial 49 finished with value: 0.5015626906757964 and parameters: {'learning_rate': 0.02382120160076752, 'num_leaves': 41, 'max_depth': 16, 'min_child_samples': 50, 'subsample': 0.6990080281515016, 'colsample_bytree': 0.7388186399441403, 'reg_alpha': 0.2094088658189187, 'reg_lambda': 0.7973638287257452}. Best is trial 34 with value: 0.5371970983176417.

Best F1 Score: 0.5371970983176417
Best Hyperparameters: {'learning_rate': 0.03215969291539223, 'num_leaves': 24, 'max_depth': 15, 'min_child_samples': 47, 'subsample': 0.8033809288783094, 'colsample_bytree': 0.8699727843770074, 'reg_alpha': 0.6751003782070576, 'reg_lambda': 0.7239832435177513}


# Analysis

1. Here we have generated `features` per filter. This preserved band specific information (eg. "color" evolution of the transient), which is clearly highly predictive

2. Used **Optuna** to find identify the best hyperparameters

3. Pushed the final score from **0.4281** to **0.5224**

Here we have likely reached the point of diminishing returns for this feature engineereing on tabular data approach. To get even higher score we need neural networks. On the next notebooks we will try Recurrent Neural Networks (RNNs) and Transformers.